In [ ]:
from agential.cog.agent.reflexion import ReflexionCoTAgent, ReflexionReActAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
)
from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

In [ ]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True
)

In [ ]:
out

In [ ]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionReActAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    max_steps=6,
    max_trials=3,
    patience=3,
    reset=True
)

In [ ]:
from agential.cog.functional.expel import gather_experience

import joblib

aa = joblib.load("C:/Users/tuvin/OneDrive/Desktop/agential/tests/assets/expel/expel_experiences_10_fake.joblib")

In [ ]:
questions = ["VIVA Media AG changed it's name in 2004. What does their new acronym stand for?",
 'Which of Jonny Craig and Pete Doherty has been a member of more bands ?',
 'Where was the first governor after the The Missouri Compromise from?',
 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
 "Woman's Era and Naj are what kind of magazines?",
 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
 "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?",
 'Which of Jonny Craig and Pete Doherty has been a member of more bands ?',
 'Where was the first governor after the The Missouri Compromise from?',
 "Woman's Era and Naj are what kind of magazines?"]

keys = ['Gesellschaft mit beschränkter Haftung',
 'Jonny" Craig',
 'Bath, Maine',
 'Creature Comforts',
 'fortnightly women interest magazine',
 'Creature Comforts',
 'Gesellschaft mit beschränkter Haftung',
 'Jonny" Craig',
 'Bath, Maine',
 'fortnightly women interest magazine']

questions = ["VIVA Media AG changed it's name in 2004. What does their new acronym stand for?",
 'Which of Jonny Craig and Pete Doherty has been a member of more bands ?',
 'Where was the first governor after the The Missouri Compromise from?',
 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
 "Woman's Era and Naj are what kind of magazines?"]

keys = ['Gesellschaft mit beschränkter Haftung',
 'Jonny" Craig',
 'Bath, Maine',
 'Creature Comforts',
 'fortnightly women interest magazine']

agent = ReflexionReActAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)

experiences = gather_experience(
    reflexion_react_agent=agent,
    questions=questions,
    keys=keys,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="reflexion",
    patience=3,
    max_trials=3
)

In [ ]:
from agential.cog.functional.expel import categorize_experiences

categorize_experiences(experiences)

In [ ]:
from agential.cog.modules.memory.expel import (
    ExpeLExperienceMemory
)

experiences = joblib.load("C:/Users/tuvin/OneDrive/Desktop/agential/tests/assets/expel/expel_experiences_10_fake.joblib")
memory = ExpeLExperienceMemory(experiences)


In [ ]:
experiences['trajectories'][3]

In [ ]:
memory.success_traj_docs

In [ ]:
experiences['trajectories'][1]

In [ ]:
joblib.dump(experiences, 'expel_experiences_10_fake.joblib')


In [ ]:
from agential.cog.prompts.benchmark.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT

import re

fewshot_questions = re.findall(r"Question: (.+?)\n", HOTPOTQA_FEWSHOT_EXAMPLES_REACT)
fewshot_keys = re.findall(
    r"Action \d+: Finish\[(.+?)\]", HOTPOTQA_FEWSHOT_EXAMPLES_REACT
)
blocks = re.split(r"(?=Question: )", HOTPOTQA_FEWSHOT_EXAMPLES_REACT)[
    1:
]  # Split and ignore the first empty result

fewshot_examples = []
for block in blocks:
    # Extract all thoughts, actions, and observations within each block
    thoughts = re.findall(r"(Thought \d+: .+?)\n", block)
    actions = re.findall(r"(Action \d+: .+?)\n", block)
    observations = re.findall(r"(Observation \d+: .+)", block)

    # Combine them into tuples and add to the examples list
    fewshot_examples.append(list(zip(thoughts, actions, observations)))

In [ ]:
len(fewshot_examples)

In [ ]:
fewshot_examples

In [ ]:
fewshot_examples[0]

In [ ]:

output_data = []
for trial in fewshot_examples[0]:
    react_output = []
    thought, action, observation = trial
    react_output.append({
        'thought': thought.split(': ', 1)[1],
        'action_type': action.split(':', 1)[0],
        'query': action.split('[', 1)[1][:-1],
        'observation': observation.split(': ', 1)[1],
        'is_correct': False  # Default to False; modify this if needed
    })
    output_data.append({
        'react_output': react_output,
        'reflections': []  # Add reflections if available
    })

In [ ]:
output_data

In [ ]:
print(blocks[0])

In [9]:
from agential.cog.prompts.benchmark.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
import re
fewshot_questions = re.findall(r"Question: (.+?)\n", HOTPOTQA_FEWSHOT_EXAMPLES_REACT)
fewshot_keys = re.findall(
    r"Action \d+: Finish\[(.+?)\]", HOTPOTQA_FEWSHOT_EXAMPLES_REACT
)

fewshot_examples = [
    [
        {
            'thought': 'I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.',
            'action_type': 'Search',
            'query': 'Colorado orogeny',
            'observation': 'The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.',
            'is_correct': False
        },
        {
            'thought': 'It does not mention the eastern sector. So I need to look up eastern sector.',
            'action_type': 'Lookup',
            'query': 'eastern sector',
            'observation': '(Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.',
            'is_correct': False
        },
        {
            'thought': 'The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.',
            'action_type': 'Search',
            'query': 'High Plains',
            'observation': 'High Plains refers to one of two distinct land regions:',
            'is_correct': False
        },
        {
            'thought': 'I need to instead search High Plains (United States).',
            'action_type': 'Search',
            'query': 'High Plains (United States)',
            'observation': 'The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]',
            'is_correct': False
        },
        {
            'thought': 'High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.',
            'action_type': 'Finish',
            'query': '1,800 to 7,000 ft',
            'observation': '1,800 to 7,000 ft',
            'is_correct': True
        }
    ] for _ in range(6)
]
from agential.cog.modules.memory.expel import (
    ExpeLExperienceMemory,)

memory = ExpeLExperienceMemory(
      fewshot_questions=fewshot_questions,
      fewshot_keys=fewshot_keys,
      fewshot_examples=fewshot_examples,
  )

In [10]:
len(fewshot_examples)

6

In [11]:
for v in memory.experiences.values():
    print(len(v))

6
6
6
6
6


In [13]:
memory.success_traj_docs

[Document(page_content='What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?', metadata={'type': 'task', 'task_idx': 0}),
 Document(page_content='Action: Search[Colorado orogeny]', metadata={'type': 'action', 'task_idx': 0}),
 Document(page_content='Action: Lookup[eastern sector]', metadata={'type': 'action', 'task_idx': 0}),
 Document(page_content='Action: Search[High Plains]', metadata={'type': 'action', 'task_idx': 0}),
 Document(page_content='Action: Search[High Plains (United States)]', metadata={'type': 'action', 'task_idx': 0}),
 Document(page_content='Action: Finish[1,800 to 7,000 ft]', metadata={'type': 'action', 'task_idx': 0}),
 Document(page_content='Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.', metadata={'type': 'thought', 'task_idx': 0}),
 Document(page_content='Thought: It does not mention the eastern sec